# 2019 MolSSI Summer School QM project: semiempirical model of Argon

## 1. Introduction

In this project, we will simulate a cluster of Argon atoms using quantum mechanics (QM) to calculate their total energy. First-principles QM simulations are complicated and expensive, and a quick implementation would rely on a substantial amount of pre-existing software infrastructure. Instead, we will implement a much simpler semiempirical QM simulation that has been designed and parameterized to reproduce first-principles QM data using a minimal model. We can then limit our external dependencies to the standard numerical functionality of Python:

In [1]:
import numpy as np

As is typically the case in quantum chemistry, we will treat the atomi nuclei as classical point charges,

$$ \vec{r}_i = (x_i , y_i, z_i). $$

While we would normally read these from a file or pass them in as an argument to a function, we will just specify a set of atomic coordinates in this example:

In [2]:
atomic_coordinates = np.array([ [0.0,0.0,0.0], [3.0,4.0,5.0] ])
number_of_atoms = len(atomic_coordinates)

np.set_printoptions(precision=1)
print(number_of_atoms)
print(atomic_coordinates)

2
[[0. 0. 0.]
 [3. 4. 5.]]


Similarly, we will only calculate and print the total energy within this notebook, whereas more useful software would probably print output data to a file or return it from a function call. You will add such features as you refactor this software. All physical quantities in this project will be specified in Hartree atomic units, where the bohr is the unit of length and the hartree is the unit of energy.

## 2. Model Hamiltonian

As is standard in quantum chemistry, we will assume that the total energy of our system is defined to be the ground state energy of a quantum many-body Hamiltonian $\hat{H}$. In second quantization notation, we can write it as

$$ \hat{H} = E_{\mathrm{ion}} + \sum_{p,q} \sum_{\sigma \in \{ \uparrow , \downarrow \} } h_{p,q} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{q,\sigma} + \tfrac{1}{2}\sum_{p,q,r,s} \sum_{\sigma,\sigma' \in \{ \uparrow , \downarrow \} } V_{p,q,r,s} \hat{a}_{p,\sigma}^{\dagger} \hat{a}_{r,\sigma'}^{\dagger} \hat{a}_{s,\sigma'} \hat{a}_{q,\sigma} , $$

where $\hat{a}_{p,\sigma}^{\dagger}$ and $\hat{a}_{p,\sigma}$ are the electron raising and lowering operators for an atomic orbital index $p$ and spin $\sigma$. We will not be using $\hat{H}$ itself in our calculations, but we will make use of the coefficient tensors $h_{p,q}$ and $V_{p,q,r,s}$. In first-principles calculations, each element of $h_{p,q}$ and $V_{p,q,r,s}$ would require the evaluation of a complicated integral. In our semiempirical model, we will set most of them to zero and assign a simple analytical form to the rest of them. The notation being used here is mostly consistent with modern quantum chemistry notation, but some objects, particularly $V_{p,q,r,s}$, have multiple conventions in practice.

### A. Model design & parameters

This semiempirical model combines some standard concepts and methods used in physics and chemistry. First, it will use a minimal number of electronic degrees of freedom. Because Argon is a noble gas, it interacts primarily through London dispersion forces that are mediated by quantum dipole fluctuations. The lowest energy dipole transition is from the occupied $3p$ states to the unoccupied $4s$ state, and we will include these 4 atomic orbitals per atom. Similarly, we will use a multipole expansion to simplify electronic excitations and retain only the monopole and dipole terms, which also restricts electronic polarization to 4 degrees of freedom per atom. We will use $\{s, p_x, p_y, p_z\}$ to label both atomic orbitals and multipole moments on each atom. The nuclear charge of Argon is 18, but our model combines the nucleus and the 12 neglected electrons ($1s^2$, $2s^2$, $2p^6$, and $3s^2$) into an ionic point charge with $Z = 6$.

In [3]:
ionic_charge = 6
orbital_types = ['s', 'px' ,'py', 'pz']
orbitals_per_atom = len(orbital_types)

p_orbitals = orbital_types[1:]
print(p_orbitals)

['px', 'py', 'pz']


The atomic orbital index contains information about which atom the orbital is centered on and the type of orbital it is. We will often extract these individual pieces of information using $\mathrm{atom}(p)$ to denote the atom's index and $\mathrm{orb}(p)$ to denote the orbital type. This is the first of many instances in this project where we could either represent something as a pre-tabulate list or a function. We will always make the simpler choice, in this case functions:

In [4]:
def atom(ao_index):
    '''Returns the atom index part of an atomic orbital index.'''
    return ao_index // orbitals_per_atom

def orb(ao_index):
    '''Returns the orbital type of an atomic orbital index.'''
    orb_index = ao_index % orbitals_per_atom
    return orbital_types[orb_index]

def ao_index(atom_p,orb_p):
    '''Returns the atomic orbital index for a given atom index and orbital type.'''
    p = atom_p*orbitals_per_atom
    p += orbital_types.index(orb_p)
    return p

print("1st atom index =", atom(0))
print("1st orbital type =", orb(0))
print("ao index of s orbital on 1st atom =", ao_index(0, 's'))

1st atom index = 0
1st orbital type = s
ao index of s orbital on 1st atom = 0


We will discuss the model parameters in more detail as they are used, but it is a good idea to collect them all in a common data structure, a Python dictionary, for convenient access throughout the notebook:

In [33]:
model_parameters = { 'r_hop' : 3.1810226927827516, # hopping length scale
                     't_ss' : 0.03365982238611262, # s-s hopping energy scale
                     't_sp' : -0.029154833035109226, # s-p hopping energy scale
                     't_pp1' : -0.0804163845390335, # 1st p-p hopping energy scale
                     't_pp2' : -0.01393611496959445, # 2nd p-p hopping energy scale
                     'r_pseudo' : 2.60342991362958, # pseudopotential length scale
                     'v_pseudo' : 0.022972992186364977, # pseudopotential energy scale
                     'dipole' : 2.781629275106456, # dipole strength of s-p transition
                     'energy_s' : 3.1659446174413004, # onsite energy of s orbital
                     'energy_p' : -2.3926873325346554, # onsite energy of p orbital
                     'coulomb_s' : 0.3603533286088998, # Coulomb self-energy of monopole
                     'coulomb_p' : -0.003267991835806299 } # Coulomb self-energy of dipole

There are no parameters related to orbital overlap because all atomic orbitals are assumed to be orthogonal. The parameter values have been pre-optimized for this project, but the fitting process and reference data are both listed at the end of the project if you'd like to learn more about them.

### B. Slater-Koster tight-binding model

We will describe the kinetic energy of electrons using a simplified [Slater-Koster tight-binding method](https://en.wikipedia.org/wiki/Tight_binding). Because of the symmetry of atomic orbitals and the translational invariance of the kinetic energy operator, there are 4 distinct, distance-dependent "hopping" energies that characterize the interatomic kinetic energy between s and p orbitals:

![s-p hopping diagram](hopping_cases.png)

All other atomic orientations can be related to these cases by a change of coordinates. While it is compatible with very general functional forms, we will use a Gaussian form to simplify the model and its implementation. The distance-dependence of this simple version is controlled by a single hopping length scale $r_{\mathrm{hop}}$ and the strength of each type of hopping energy,

$$ t_{o,o'}(\vec{r}) = \exp(1-r^2/r_{\mathrm{hop}}^2) \times \begin{cases}
 t_{ss} , & o = o' = s \\
 [\vec{o}' \cdot (\vec{r}/r_{\mathrm{hop}})] t_{sp}, & o = s \ \& \ o' \in \{p_x, p_y, p_z\} \\
 -[\vec{o} \cdot (\vec{r}/r_{\mathrm{hop}})] t_{sp} , & o' = s \ \& \ o \in \{p_x, p_y, p_z\} \\
 (r^2/r_{\mathrm{SK}}^2)\,(\vec{o} \cdot \vec{o}')  t_{pp2} - [\vec{o} \cdot (\vec{r}/r_{\mathrm{SK}})] [\vec{o}' \cdot (\vec{r}/r_{\mathrm{SK}})] (t_{pp1} + t_{pp2}), & o,o' \in \{p_x, p_y, p_z\}
 \end{cases} $$
 
where $o$ and $o'$ are the orbital types of the 1st and 2nd atoms and $\vec{r}$ is a vector pointing from the 2nd atom to the 1st atom. We are assigning direction vectors to the p orbitals, $\vec{p}_x \equiv (1,0,0)$, $\vec{p}_y \equiv (0,1,0)$, and $\vec{p}_z \equiv (0,0,1)$, to simplify the notation. This project has multiple case-based formulas, and we will implement them using a code structure similar to each formula:

In [6]:
vec = { 'px':[1,0,0], 'py':[0,1,0], 'pz':[0,0,1] }

def hopping_energy(o1, o2, r12, model_parameters):
    '''Returns the hopping matrix element for a pair of orbitals of type o1 & o2 separated by a vector r12.'''
    r12_rescaled = r12 / model_parameters['r_hop']
    r12_squared = np.dot(r12_rescaled, r12_rescaled)
    ans = np.exp( 1.0 - r12_squared )
    if o1 == 's' and o2 == 's':
        ans *= model_parameters['t_ss']
    if o1 == 's' and o2 in p_orbitals:
        ans *= np.dot(vec[o2], r12_rescaled) * model_parameters['t_sp']
    if o2 == 's' and o1 in p_orbitals:
        ans *= -np.dot(vec[o1], r12_rescaled)* model_parameters['t_sp']
    if o1 in p_orbitals and o2 in p_orbitals:
        ans *= ( r12_squared * np.dot(vec[o1], vec[o2]) * model_parameters['t_pp2']
                 - np.dot(vec[o1], r12_rescaled) * np.dot(vec[o2], r12_rescaled)
                 * ( model_parameters['t_pp1'] + model_parameters['t_pp2'] ) )
    return ans

print("vec[px] =",vec['px'])
print("hopping test",hopping_energy('s','px',np.array([5.0,0.0,0.0]),model_parameters))

vec[px] = [1, 0, 0]
hopping test 0.0


### C. Coulomb interaction

For the purpose of electrostatics, we will describe all inter-atomic Coulomb interactions with point charges and all intra-atomic Coulomb interactions between electrons with electronic self-energy parameters $V_o^{\mathrm{self}}$ for multipole moment $o$. We need access to both the interaction kernel and its derivatives to define the multipole expansion,

$$ V_{o,o'}(\vec{r}) = \begin{cases}
 1/r , & o = o' = s \\
 (\vec{o}' \cdot \vec{r}) / r^3, & o = s \ \& \ o' \in \{p_x, p_y, p_z\} \\
 -(\vec{o} \cdot \vec{r}) / r^3 , & o' = s \ \& \ o \in \{p_x, p_y, p_z\} \\
 (\vec{o} \cdot \vec{o}') / r^3 - 3 (\vec{o} \cdot \vec{r}) (\vec{o}' \cdot \vec{r}) / r^5, & o,o' \in \{p_x, p_y, p_z\}
 \end{cases} $$

In [7]:
def coulomb_energy(o1, o2, r12):
    '''Returns the Coulomb matrix element for a pair of multipoles of type o1 & o2 separated by a vector r12.'''
    r12_length = np.linalg.norm(r12)
    if o1 == 's' and o2 == 's':
        ans = 1.0 / r12_length
    if o1 == 's' and o2 in p_orbitals:
        ans = np.dot(vec[o2], r12) / r12_length**3
    if o2 == 's' and o1 in p_orbitals:
        ans = -np.dot(vec[o1], r12) / r12_length**3
    if o1 in p_orbitals and o2 in p_orbitals:
        ans = ( np.dot(vec[o1], vec[o2]) / r12_length**3
               - 3.0 * np.dot(vec[o1], r12) * np.dot(vec[o2], r12) / r12_length**5 )
    return ans 

The semiempirical model approximations strongly distort the physics of inter-atomic Pauli repulsion, and we compensate for these errors with a short-range ionic pseudopotential, which is a common tool in physics for building effective models of ionic cores:

$$ V_o^{\mathrm{pseudo}}(\vec{r}) = V_{\mathrm{pseudo}} \exp(1 - r^2/r_{\mathrm{pseudo}}^2) \times \begin{cases}
1, & o = s \\
-2 \vec{o}\cdot\vec{r}/r_{\mathrm{pseudo}}, & o \in \{p_x, p_y, p_z\}. \end{cases} $$

In [8]:
def pseudopotential_energy(o, r, model_parameters):
    '''Returns the energy of a pseudopotential between a multipole of type o and an atom separated by a vector r.'''
    ans = model_parameters['v_pseudo']
    r_rescaled = r / model_parameters['r_pseudo']
    ans *= np.exp( 1.0 - np.dot(r_rescaled,r_rescaled) )
    if o in p_orbitals:
        ans *= -2.0 * np.dot(vec[o], r_rescaled)
    return ans

These interaction kernels enable us to define and calculate the ion-ion energy in $\hat{H}$,

$$ E_{\mathrm{ion}} = Z^2 \sum_{i < j} V_{s,s}(\vec{r}_i - \vec{r}_j) $$

In [9]:
def calculate_energy_ion(atomic_coordinates):
    '''Returns the ionic contribution to the total energy for an input list of atomic coordinates.'''
    energy_ion = 0.0
    for i, r_i in enumerate(atomic_coordinates):
        for j, r_j in enumerate(atomic_coordinates):
            if i < j:
                energy_ion += (ionic_charge**2)*coulomb_energy('s', 's', r_i - r_j)
    return energy_ion

energy_ion = calculate_energy_ion(atomic_coordinates)
print(energy_ion)

5.091168824543142


the vector of electron-ion interactions,

$$ V^{\mathrm{ion}}_p = \sum_{i \neq \mathrm{atom}(p)}
   V_{\mathrm{orb}(p)}^{\mathrm{pseudo}}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_i)
   - Z V_{\mathrm{orb}(p),s}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_i)$$

In [10]:
def calculate_potential_vector(atomic_coordinates, model_parameters):
    '''Returns the electron-ion potential energy vector for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    potential_vector = np.zeros(ndof)
    for p in range(ndof):
        potential_vector[p] = 0.0
        for atom_i,r_i in enumerate(atomic_coordinates):
            r_pi = atomic_coordinates[atom(p)] - r_i
            if atom_i != atom(p):
                potential_vector[p] += ( pseudopotential_energy(orb(p), r_pi, model_parameters) 
                                         - ionic_charge * coulomb_energy(orb(p), 's', r_pi) )
    return potential_vector

potential_vector = calculate_potential_vector(atomic_coordinates, model_parameters)
print(potential_vector)

[-0.8 -0.1 -0.1 -0.1 -0.8  0.1  0.1  0.1]


and the matrix of electron-electron interaction matrix elements,

$$ V^{\mathrm{ee}}_{p,q} = \begin{cases}
 V_{\mathrm{orb}(p),\mathrm{orb}(q)}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)}) , & \mathrm{atom}(p) \neq \mathrm{atom}(q) \\
 V^{\mathrm{self}}_{\mathrm{orb}(p)} \delta_{p,q} , & \mathrm{atom}(p) = \mathrm{atom}(q)
\end{cases} . $$

In [11]:
def calculate_interaction_matrix(atomic_coordinates, model_parameters):
    '''Returns the electron-electron interaction energy matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    interaction_matrix = np.zeros( (ndof,ndof) )
    for p in range(ndof):
        for q in range(ndof):
            if atom(p) != atom(q):
                r_pq = atomic_coordinates[atom(p)] - atomic_coordinates[atom(q)]
                interaction_matrix[p,q] = coulomb_energy(orb(p), orb(q), r_pq)
            if p == q and orb(p) == 's':
                interaction_matrix[p,q] = model_parameters['coulomb_s']
            if p == q and orb(p) in p_orbitals:
                interaction_matrix[p,q] = model_parameters['coulomb_p']                
    return interaction_matrix

interaction_matrix = calculate_interaction_matrix(atomic_coordinates, model_parameters)
print(interaction_matrix)

[[ 3.6e-01  0.0e+00  0.0e+00  0.0e+00  1.4e-01 -8.5e-03 -1.1e-02 -1.4e-02]
 [ 0.0e+00  3.0e-03  0.0e+00  0.0e+00  8.5e-03  1.3e-03 -2.0e-03 -2.5e-03]
 [ 0.0e+00  0.0e+00  3.0e-03  0.0e+00  1.1e-02 -2.0e-03  1.1e-04 -3.4e-03]
 [ 0.0e+00  0.0e+00  0.0e+00  3.0e-03  1.4e-02 -2.5e-03 -3.4e-03 -1.4e-03]
 [ 1.4e-01  8.5e-03  1.1e-02  1.4e-02  3.6e-01  0.0e+00  0.0e+00  0.0e+00]
 [-8.5e-03  1.3e-03 -2.0e-03 -2.5e-03  0.0e+00  3.0e-03  0.0e+00  0.0e+00]
 [-1.1e-02 -2.0e-03  1.1e-04 -3.4e-03  0.0e+00  0.0e+00  3.0e-03  0.0e+00]
 [-1.4e-02 -2.5e-03 -3.4e-03 -1.4e-03  0.0e+00  0.0e+00  0.0e+00  3.0e-03]]


### D. Multipole decomposition

To define $V_{p,q,r,s}$ based on the Coulomb matrix elements $V_{p,q}^{\mathrm{ee}}$, we need to define a mapping from products of atomic orbitals to a linear combination of terms in the multipole expansion of electronic charge. Because the atomic orbitals are normalized, their monopole coefficient with themselves is 1 (i.e. they have unit charge). Because of orthogonality, there is no monopole term for either intra-atomic or inter-atomic transitions between atomic orbitals. We will ignore dipole transitions between atoms, which corresponds to the [neglect of diatomic differential overlap](https://en.wikipedia.org/wiki/NDDO) (NDDO) approximation that is commonly used in semiempirical quantum chemistry. All that remains is the intra-atomic s-p transition, and we will use a model parameter, $D$, to define its dipole strength. These transformation rules between atomic orbitals and multipole moments can be summarized pictorially,

![s-p multipole diagram](multipole_cases.png)

or mathematically as a 3-index tensor $\chi_{p,q,r}$ where $p$ and $q$ are the atomic orbital indices and $r$ is the multipole moment index,

$$ \chi_{p, q, r} = \begin{cases} 1, & \mathrm{orb}(p) = \mathrm{orb}(q) \ \& \ \mathrm{orb}(r) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\ 
 D , & \mathrm{orb}(q) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(p) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 D , & \mathrm{orb}(p) = \mathrm{orb}(r) \in \{p_x, p_y, p_z\} \ \& \ \mathrm{orb}(q) = s \ \& \ \mathrm{atom}(p) = \mathrm{atom}(q) = \mathrm{atom}(r) \\
 0, & \mathrm{otherwise} \end{cases} . $$

In [12]:
def chi_on_atom(o1, o2, o3, model_parameters):
    '''Returns the value of the chi tensor for 3 orbital indices on the same atom.'''
    if o1 == o2 and o3 == 's':
        return 1.0
    if o1 == o3 and o3 in p_orbitals and o2 == 's':
        return model_parameters['dipole']
    if o2 == o3 and o3 in p_orbitals and o1 == 's':
        return model_parameters['dipole']
    return 0.0

def calculate_chi_tensor(atomic_coordinates, model_parameters):
    '''Returns the chi tensor for an input list of atomic coordinates'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    chi_tensor = np.zeros( (ndof,ndof,ndof) )
    for p in range(ndof):
        for orb_q in orbital_types:
            q = ao_index(atom(p), orb_q) # p & q on same atom
            for orb_r in orbital_types:
                r = ao_index(atom(p), orb_r) # p & r on same atom
                chi_tensor[p,q,r] = chi_on_atom(orb(p), orb(q), orb(r), model_parameters)
    return chi_tensor

chi_tensor = calculate_chi_tensor(atomic_coordinates, model_parameters)
print(chi_tensor)

[[[1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 

The multipole expansion plays the same role as an auxiliary basis set in modern resolution-of-identity (RI) methods that are used to accelerate large quantum chemistry simulations. The primary purpose of these methods is to decompose $V_{p,q,r,s}$ into a low-rank factored form, which is

$$ V_{p,q,r,s} = \sum_{t,u} \chi_{p,q,t} V_{t,u}^{\mathrm{ee}} \chi_{r,s,u} $$

in our semiempirical model. Unlike the previous vectors and matrices that we have constructed, the $\chi$ and $V$ tensors are sparse, meaning that most of their entries are zero. Without a more clever implementation, it can be computationally slow and wasteful to store and compute with these zero values. Modern quantum chemistry methods contain many ways of identifying and utilizing sparsity. For example, we could compute tensor elements of $\chi$ on-the-fly rather than storing the full tensor. This is analogous to "integral direct" methods in quantum chemistry. For simplicity, we will still precompute and store $\chi$, which fails to utilize its sparsity. However, we will avoid the explicit construction of $V_{p,q,r,s}$ by utilizing its factored form.

### E. 1-body Hamiltonian

The 1-body Hamiltonian coefficients $h_{p,q}$ combine many of the components that we have already discussed and implemented along with the on-site orbital energies, $E_s$ and $E_p$, that are the final two parameters from the semiempirical model,

$$ h_{p,q} = \begin{cases}
 t_{\mathrm{orb}(p),\mathrm{orb}(q)}(\vec{r}_{\mathrm{atom}(p)} - \vec{r}_{\mathrm{atom}(q)})
  , & \mathrm{atom}(p) \neq \mathrm{atom}(q) \\
 E_{\mathrm{orb}(p)} \delta_{\mathrm{orb}(p),\mathrm{orb}(q)} + \sum_{r} \chi_{p,q,r} V_{r}^{\mathrm{ion}} , & \mathrm{atom}(p) = \mathrm{atom}(q) 
  \end{cases}. $$

In [13]:
def calculate_hamiltonian_matrix(atomic_coordinates, model_parameters):
    '''Returns the 1-body Hamiltonian matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    hamiltonian_matrix = np.zeros( (ndof,ndof) )
    potential_vector = calculate_potential_vector(atomic_coordinates, model_parameters)
    for p in range(ndof):
        for q in range(ndof):
            if atom(p) != atom(q):
                r_pq = atomic_coordinates[atom(p)] - atomic_coordinates[atom(q)]
                hamiltonian_matrix[p,q] = hopping_energy(orb(p), orb(q), r_pq, model_parameters)
            if atom(p) == atom(q):
                if p == q and orb(p) == 's':
                    hamiltonian_matrix[p,q] += model_parameters['energy_s']
                if p == q and orb(p) in p_orbitals:
                    hamiltonian_matrix[p,q] += model_parameters['energy_p']
                for orb_r in orbital_types:
                    r = ao_index(atom(p), orb_r)
                    hamiltonian_matrix[p,q] += ( chi_on_atom(orb(p), orb(q), orb_r, model_parameters)
                                                 * potential_vector[r] )
    return hamiltonian_matrix

hamiltonian_matrix = calculate_hamiltonian_matrix(atomic_coordinates, model_parameters)
print("hamiltonian matrix =\n",hamiltonian_matrix)

hamiltonian matrix =
 [[-6.0e-01 -5.2e-02 -6.9e-02 -8.6e-02 -2.0e-04 -0.0e+00 -0.0e+00 -0.0e+00]
 [-5.2e-02 -3.2e+00  0.0e+00  0.0e+00  0.0e+00  2.7e-04  2.2e-03  2.7e-03]
 [-6.9e-02  0.0e+00 -3.2e+00  0.0e+00  0.0e+00  2.2e-03  1.5e-03  3.7e-03]
 [-8.6e-02  0.0e+00  0.0e+00 -3.2e+00  0.0e+00  2.7e-03  3.7e-03  3.2e-03]
 [-2.0e-04  0.0e+00  0.0e+00  0.0e+00 -6.0e-01  5.2e-02  6.9e-02  8.6e-02]
 [-0.0e+00  2.7e-04  2.2e-03  2.7e-03  5.2e-02 -3.2e+00  0.0e+00  0.0e+00]
 [-0.0e+00  2.2e-03  1.5e-03  3.7e-03  6.9e-02  0.0e+00 -3.2e+00  0.0e+00]
 [-0.0e+00  2.7e-03  3.7e-03  3.2e-03  8.6e-02  0.0e+00  0.0e+00 -3.2e+00]]


We have now fully specified the many-body Hamiltonian $\hat{H}$, and we can now move on to approximating and calculating some of its physical properties.

## 3. Hartree-Fock theory

Even with a simple model for $\hat{H}$, we cannot calculate its ground state energy exactly for more than a few Argon atoms. Instead, we will use the [Hartree-Fock approximation](https://en.wikipedia.org/wiki/Hartree–Fock_method), which restricts the ground-state wavefunction to a single Slater determinant. We will find the Slater determinant with the lowest total energy, but a more general wavefunction will usually have an even lower energy. In the next section, we will use many-body perturbation theory to improve our estimate of the total energy.

The central objects of Hartree-Fock theory are the 1-electron density matrix $\rho_{p,q}$ and the Fock matrix $f_{p,q}$. These two matrices depend on each other, which defines a nonlinear set of equations that we must solve iteratively. Iteratively solving these equations is usually referred to as the self-consistent field (SCF) cycle. For this to converge, we must start from a reasonable initial guess for $\rho_{p,q}$. We will initialize it to the density matrix for isolated Argon atoms,

$$ \rho_{p,q}^{\mathrm{atom}} = \begin{cases} 
   1, & p = q \ \& \ \mathrm{orb}(p) \in \{ p_x, p_y, p_z \} \\
   0, & \mathrm{otherwise}
\end{cases} $$

In [14]:
orbital_occupation = { 's':0, 'px':1, 'py':1, 'pz':1 }

def calculate_atomic_density_matrix(atomic_coordinates):
    '''Returns a trial 1-electron density matrix for an input list of atomic coordinates.'''
    ndof = len(atomic_coordinates)*orbitals_per_atom
    density_matrix = np.zeros( (ndof,ndof) )
    for p in range(ndof):
        density_matrix[p,p] = orbital_occupation[orb(p)]
    return density_matrix

density_matrix = calculate_atomic_density_matrix(atomic_coordinates)
print(density_matrix)

print(2.0*np.einsum('pq,pq',hamiltonian_matrix,density_matrix))

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
-38.98189299179223


Because of spin symmetry, we use the same $\rho_{p,q}$ for each electron spin type, which reduces the sum over spin types in $\hat{H}$ to degeneracy pre-factors. Thus, half of the electrons are spin-up and half are spin-down.

The Fock matrix is defined by the density matrix,

$$ \begin{align} f_{p,q} & = h_{p,q} + \sum_{r,s} ( 2 V_{p,q,r,s} - V_{r,q,p,s} )\rho_{r,s} \\
                         & = h_{p,q} + \sum_{r,s,t,u} ( 2 \chi_{p,q,t} \chi_{r,s,u} - \chi_{r,q,t} \chi_{p,s,u} )
                             V_{t,u}^{\mathrm{ee}} \rho_{r,s}
   \end{align} $$

In [15]:
def calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor):
    '''Returns the Fock matrix defined by the input Hamiltonian, interaction, & density matrices.'''
    fock_matrix = hamiltonian_matrix.copy()
    fock_matrix += 2.0*np.einsum('pqt,rsu,tu,rs',
                                 chi_tensor, chi_tensor, interaction_matrix, density_matrix, optimize=True)
    fock_matrix -= np.einsum('rqt,psu,tu,rs',
                             chi_tensor, chi_tensor, interaction_matrix, density_matrix, optimize=True)
    return fock_matrix

fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
print(fock_matrix)

[[ 2.4e+00  8.7e-05  1.2e-04  1.4e-04 -2.0e-04  0.0e+00  0.0e+00  0.0e+00]
 [ 8.7e-05 -5.8e-01  0.0e+00  0.0e+00  0.0e+00  2.7e-04  2.2e-03  2.7e-03]
 [ 1.2e-04  0.0e+00 -5.8e-01  0.0e+00  0.0e+00  2.2e-03  1.5e-03  3.7e-03]
 [ 1.4e-04  0.0e+00  0.0e+00 -5.8e-01  0.0e+00  2.7e-03  3.7e-03  3.2e-03]
 [-2.0e-04  0.0e+00  0.0e+00  0.0e+00  2.4e+00 -8.7e-05 -1.2e-04 -1.4e-04]
 [ 0.0e+00  2.7e-04  2.2e-03  2.7e-03 -8.7e-05 -5.8e-01  0.0e+00  0.0e+00]
 [ 0.0e+00  2.2e-03  1.5e-03  3.7e-03 -1.2e-04  0.0e+00 -5.8e-01  0.0e+00]
 [ 0.0e+00  2.7e-03  3.7e-03  3.2e-03 -1.4e-04  0.0e+00  0.0e+00 -5.8e-01]]


which is just a sum of two tensor contractions between the $V$ tensor and the density matrix. Because the $V$ tensor has low rank, we can save a lot of computational effort by using its factored form. This is our first example of using "einsum" in NumPy to trivialize the implementation of a tensor equation.

The Fock matrix defines the matrix of molecular orbitals $\phi_{i,j}$ as its eigenvectors,

$$ \sum_{q} f_{p,q} \phi_{q,r} = E_r \phi_{p,r}, $$

and the density matrix is defined by the occupied orbitals (we occupy the lowest-energy orbitals with all of the electrons available),

$$ \rho_{p,q} = \sum_{i \in \mathrm{occ}} \phi_{p,i} \phi_{q,i}. $$

In [16]:
def calculate_density_matrix(fock_matrix):
    '''Returns the 1-electron density matrix defined by the input Fock matrix.'''
    num_occ = (ionic_charge//2)*np.size(fock_matrix,0)//orbitals_per_atom
    orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix)
    occupied_matrix = orbital_matrix[:,:num_occ]
    density_matrix = occupied_matrix @ occupied_matrix.T
    return density_matrix

density_matrix = calculate_density_matrix(fock_matrix)
print(density_matrix)

[[ 4.7e-09 -2.9e-05 -3.9e-05 -4.8e-05 -2.4e-11 -7.3e-08 -9.7e-08 -1.2e-07]
 [-2.9e-05  1.0e+00 -1.1e-09 -1.4e-09  7.3e-08 -4.2e-12 -5.6e-12 -7.1e-12]
 [-3.9e-05 -1.1e-09  1.0e+00 -1.9e-09  9.7e-08 -5.6e-12 -7.5e-12 -9.4e-12]
 [-4.8e-05 -1.4e-09 -1.9e-09  1.0e+00  1.2e-07 -7.1e-12 -9.4e-12 -1.2e-11]
 [-2.4e-11  7.3e-08  9.7e-08  1.2e-07  4.7e-09  2.9e-05  3.9e-05  4.8e-05]
 [-7.3e-08 -4.2e-12 -5.6e-12 -7.1e-12  2.9e-05  1.0e+00 -1.1e-09 -1.4e-09]
 [-9.7e-08 -5.6e-12 -7.5e-12 -9.4e-12  3.9e-05 -1.1e-09  1.0e+00 -1.9e-09]
 [-1.2e-07 -7.1e-12 -9.4e-12 -1.2e-11  4.8e-05 -1.4e-09 -1.9e-09  1.0e+00]]


To calculate a $\rho_{p,q}$ and $f_{p,q}$ that are consistent with each other, we mix in a fraction of the new density matrix with the old density matrix and re-calculate them until convergence. This is a crude strategy for an SCF cycle, but it works for weakly interacting atoms like Argon:

In [17]:
def scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor):
    '''Returns converged density & Fock matrices defined by the input Hamiltonian, interaction, & density matrices.'''
    MAX_SCF_ITERATIONS = 100
    MIXING_FRACTION = 0.25
    CONVERGENCE_TOLERANCE = 1e-4
    for iteration in range(MAX_SCF_ITERATIONS):
        fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
        new_density_matrix = calculate_density_matrix(fock_matrix)

        error_norm = np.linalg.norm( density_matrix - new_density_matrix )
        if error_norm < CONVERGENCE_TOLERANCE:
            return density_matrix, fock_matrix

        density_matrix = (MIXING_FRACTION*new_density_matrix
                          + (1.0 - MIXING_FRACTION)*density_matrix)
    print("SCF cycle didn't converge")
    return density_matrix, fock_matrix

density_matrix, fock_matrix = scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
print("density matrix after SCF:\n",density_matrix)

density matrix after SCF:
 [[ 4.7e-09 -2.9e-05 -3.9e-05 -4.8e-05 -2.4e-11 -7.3e-08 -9.7e-08 -1.2e-07]
 [-2.9e-05  1.0e+00 -1.1e-09 -1.4e-09  7.3e-08 -4.2e-12 -5.6e-12 -7.1e-12]
 [-3.9e-05 -1.1e-09  1.0e+00 -1.9e-09  9.7e-08 -5.6e-12 -7.5e-12 -9.4e-12]
 [-4.8e-05 -1.4e-09 -1.9e-09  1.0e+00  1.2e-07 -7.1e-12 -9.4e-12 -1.2e-11]
 [-2.4e-11  7.3e-08  9.7e-08  1.2e-07  4.7e-09  2.9e-05  3.9e-05  4.8e-05]
 [-7.3e-08 -4.2e-12 -5.6e-12 -7.1e-12  2.9e-05  1.0e+00 -1.1e-09 -1.4e-09]
 [-9.7e-08 -5.6e-12 -7.5e-12 -9.4e-12  3.9e-05 -1.1e-09  1.0e+00 -1.9e-09]
 [-1.2e-07 -7.1e-12 -9.4e-12 -1.2e-11  4.8e-05 -1.4e-09 -1.9e-09  1.0e+00]]


Once we have a converged density matrix, the Hartree-Fock total energy is defined as

$$ \begin{align}
  E_{\mathrm{HF}} &= E_{\mathrm{ion}} + E_{\mathrm{SCF}} \\
  E_{\mathrm{SCF}} &= 2 \sum_{p,q} h_{p,q} \rho_{p,q}
  + \sum_{p,q,r,s} V_{p,q,r,s} ( 2 \rho_{p,q} \rho_{r,s} - \rho_{p,s} \rho_{r,q} ) = \sum_{p,q} (h_{p,q} + f_{p,q}) \rho_{p,q}
  \end{align} $$

In [40]:
def calculate_energy_scf(hamiltonian_matrix, fock_matrix, density_matrix):
    '''Returns the Hartree-Fock total energy defined by the input Hamiltonian, Fock, & density matrices.'''
    energy_scf = np.einsum('pq,pq',hamiltonian_matrix + fock_matrix,density_matrix)
    return energy_scf

energy_scf = calculate_energy_scf(hamiltonian_matrix, fock_matrix, density_matrix)
print("Hartree-Fock energy =",energy_ion + energy_scf)

Hartree-Fock energy = -2507.535082912496


The bottleneck of this Hartree-Fock implementation is the tensor contraction in the formation of the Fock matrix. Because we are not exploiting the sparsity of $\chi$, it scales as $O(n^4)$ operations for $n$ atoms because of 3 free tensor indices and 1 summed tensor index in the contractions with the dense $\chi$ tensor.

## 4. 2nd-order Moller-Plesset (MP2) perturbation theory

The Hartree-Fock approximation does not describe the London dispersion interaction between Argon atoms. We must use many-body perturbation theory to improve our approximation to the ground state of $\hat{H}$. Thankfully, we only need to use the first correction beyond Hartree-Fock theory, which is [second-order Moller-Plesset (MP2) perturbation theory](https://en.wikipedia.org/wiki/Møller–Plesset_perturbation_theory). It is simple, but computationally expensive. The computational bottleneck is the transformation of the tensor $V_{p,q,r,s}$ from the atomic orbital basis to the molecular orbital basis,

$$ \tilde{V}_{a,i,b,j} = \sum_{p,q,r,s} \phi_{p,a} \phi_{q,i} \phi_{r,b} \phi_{s,j} V_{p,q,r,s}. $$

where we are restricting some indices to occupied orbitals ($i$ and $j$) and some indices to the unoccupied "virtual" orbitals ($a$ and $b$). This use of specific orbital labels to denote a restriction of indices is standard notation in quantum chemistry. To make this easier to program, we can partition the molecular orbitals and their energies into occupied and virtual:

In [19]:
def partition_orbitals(fock_matrix):
    '''Returns a list with the occupied/virtual energies & orbitals defined by the input Fock matrix.'''
    num_occ = (ionic_charge//2)*np.size(fock_matrix,0)//orbitals_per_atom
    orbital_energy, orbital_matrix = np.linalg.eigh(fock_matrix)
    occupied_energy = orbital_energy[:num_occ]
    virtual_energy = orbital_energy[num_occ:]
    occupied_matrix = orbital_matrix[:,:num_occ]
    virtual_matrix = orbital_matrix[:,num_occ:]

    return occupied_energy, virtual_energy, occupied_matrix, virtual_matrix

occupied_energy, virtual_energy, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)

np.set_printoptions(precision=6)
print(f'occupied orbital energies:\n {occupied_energy}')
print("virtual orbital energies:\n", virtual_energy)

occupied orbital energies:
 [-0.592721 -0.586342 -0.586342 -0.583595 -0.583595 -0.577216]
virtual orbital energies:
 [2.412468 2.412868]


We use the factored form of $V_{p,q,r,s}$ to reduce our intermediate memory usage, but we do need to explicitly store $\tilde{V}_{a,i,b,j}$,

$$ \begin{align}
   \tilde{\chi}_{a,i,p} &= \sum_{q,r} \phi_{q,a} \phi_{r,i} \chi_{q,r,p} \\
   \tilde{V}_{a,i,b,j} &= \sum_{p,q} \tilde{\chi}_{a,i,p} V_{p,q}^{\mathrm{ee}} \tilde{\chi}_{b,j,q}
   \end{align} $$

In [20]:
def transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix, chi_tensor):
    '''Returns a transformed V tensor defined by the input occupied, virtual, & interaction matrices.'''
    chi2_tensor = np.einsum('qa,ri,qrp', virtual_matrix, occupied_matrix, chi_tensor, optimize=True)
    interaction_tensor = np.einsum('aip,pq,bjq->aibj', chi2_tensor, interaction_matrix, chi2_tensor, optimize=True)
    return interaction_tensor

interaction_tensor = transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix, chi_tensor)
print(interaction_tensor)

[[[[ 4.503295e-03  1.809314e-17  3.921942e-17 -3.085862e-17
    -4.908258e-17 -4.084409e-15]
   [ 7.760915e-15 -4.937673e-17  7.459311e-17 -3.102329e-17
    -1.892930e-16  4.503295e-03]]

  [[ 1.822413e-17  3.031948e-03 -8.163704e-19 -3.408466e-15
     9.462420e-16  4.935618e-17]
   [ 2.814967e-17  3.264893e-15  4.288329e-17  2.940322e-03
    -7.397402e-04  1.929257e-17]]

  [[ 3.927959e-17 -7.728544e-19  3.031948e-03 -1.014768e-15
    -3.853600e-15 -6.992321e-17]
   [ 1.498007e-17 -8.839736e-17  3.740578e-15  7.397402e-04
     2.940322e-03  1.914104e-16]]

  [[-3.062340e-17 -3.408490e-15 -1.014906e-15  8.925221e-05
     9.902406e-20 -8.697126e-18]
   [-9.827973e-18  8.655499e-05  2.177592e-05 -3.282291e-15
    -1.579464e-16 -3.062194e-17]]

  [[-4.928376e-17  9.463395e-16 -3.853740e-15  4.551368e-20
     8.925221e-05  4.003643e-17]
   [ 3.570628e-17 -2.177592e-05  8.655499e-05 -2.778647e-17
    -3.722549e-15 -5.185197e-17]]

  [[-4.084169e-15  4.954015e-17 -7.003368e-17 -8.715413e-18


In this simple implementation, we are not taking full advantage of the sparisty of the initial $\chi$ tensor. Unlike the $V$ tensor, the $\tilde{V}$ tensor is constructed explicitly and stored in memory. The MP2 correction to the total energy is then just a large summation over previously computed and stored quantities,

$$ E_{\mathrm{MP2}} = - \sum_{a,b \in \mathrm{virt}} \sum_{i,j \in \mathrm{occ}} \frac{2 \tilde{V}_{a,i,b,j}^2 - \tilde{V}_{a,i,b,j} \tilde{V}_{a,j,b,i}}{E_a + E_b - E_i - E_j} . $$

In [21]:
def calculate_energy_mp2(fock_matrix, interaction_matrix, chi_tensor):
    '''Returns the MP2 contribution to the total energy defined by the input Fock & interaction matrices.'''
    E_occ, E_virt, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)
    V_tilde = transform_interaction_tensor(occupied_matrix, virtual_matrix, interaction_matrix, chi_tensor)

    energy_mp2 = 0.0
    num_occ = len(E_occ)
    num_virt = len(E_virt)
    for a in range(num_virt):
        for b in range(num_virt):
            for i in range(num_occ):
                for j in range(num_occ):
                    energy_mp2 -= ( (2.0*V_tilde[a,i,b,j]**2 - V_tilde[a,i,b,j]*V_tilde[a,j,b,i])
                                    /(E_virt[a] + E_virt[b] - E_occ[i] - E_occ[j]) )
    return energy_mp2

energy_mp2 = calculate_energy_mp2(fock_matrix, interaction_matrix, chi_tensor)
print(energy_mp2)

-4.441475799149136e-05


The bottleneck of MP2 calculations is the formation of $\tilde{V}$ from $\tilde{\chi}$ and $V^{\mathrm{ee}}$, which scales as $O(n^5)$ operations for $n$ atoms because of 4 free tensor indices and 1 summed tensor index once the contraction has been optimally arranged into intermediate operations.

## (Extra Credit) 5. Alternative Fock matrix construction

The previous implementation of Fock matrix construction,

$$ f_{p,q} = h_{p,q} + \sum_{r,s,t,u} ( 2 \chi_{p,q,t} \chi_{r,s,u} - \chi_{r,q,t} \chi_{p,s,u} )
                             V_{t,u}^{\mathrm{ee}} \rho_{r,s}, $$

did not utilize the sparsity of the $\chi$ tensor. As a result, its computational cost scaling is not as low as it could be, and it will have poor performance for large system sizes. We can improve the scaling by making use of the fact that $\chi_{p,q,r}$ is only non-zero when all of its indices are on the same atom:

In [22]:
def calculate_fock_matrix_fast(hamiltonian_matrix, interaction_matrix, density_matrix, model_parameters):
    '''Returns the Fock matrix defined by the input Hamiltonian, interaction, & density matrices.'''
    ndof = np.size(hamiltonian_matrix,0)
    fock_matrix = hamiltonian_matrix.copy()
    # Hartree potential term
    for p in range(ndof):
        for orb_q in orbital_types:
            q = ao_index(atom(p), orb_q) # p & q on same atom
            for orb_t in orbital_types:
                t = ao_index(atom(p), orb_t) # p & t on same atom
                chi_pqt = chi_on_atom(orb(p), orb_q, orb_t, model_parameters)
                for r in range(ndof):
                    for orb_s in orbital_types:
                        s = ao_index(atom(r), orb_s) # r & s on same atom
                        for orb_u in orbital_types:
                            u = ao_index(atom(r), orb_u) # r & u on same atom
                            chi_rsu = chi_on_atom(orb(r), orb_s, orb_u, model_parameters)
                            fock_matrix[p,q] += 2.0 * chi_pqt * chi_rsu * interaction_matrix[t,u] * density_matrix[r,s]
    # Fock exchange term
    for p in range(ndof):
        for orb_s in orbital_types:
            s = ao_index(atom(p), orb_s) # p & s on same atom
            for orb_u in orbital_types:
                u = ao_index(atom(p), orb_u) # p & u on same atom
                chi_psu = chi_on_atom(orb(p), orb_s, orb_u, model_parameters)
                for q in range(ndof):
                    for orb_r in orbital_types:
                        r = ao_index(atom(q), orb_r) # q & r on same atom
                        for orb_t in orbital_types:
                            t = ao_index(atom(q), orb_t) # q & t on same atom
                            chi_rqt = chi_on_atom(orb_r, orb(q), orb_t, model_parameters)
                            fock_matrix[p,q] -= chi_rqt * chi_psu * interaction_matrix[t,u] * density_matrix[r,s]
    return fock_matrix

fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
print(fock_matrix)
fock_matrix = calculate_fock_matrix_fast(hamiltonian_matrix, interaction_matrix, density_matrix, model_parameters)
print(fock_matrix)

[[ 2.412668e+00  9.645498e-05  1.286066e-04  1.607583e-04 -1.997171e-04
   1.075730e-08  1.434306e-08  1.792883e-08]
 [ 9.645498e-05 -5.849685e-01  4.066361e-10  5.082951e-10 -1.075730e-08
   2.688693e-04  2.190197e-03  2.737746e-03]
 [ 1.286066e-04  4.066361e-10 -5.849685e-01  6.777269e-10 -1.434306e-08
   2.190197e-03  1.546484e-03  3.650328e-03]
 [ 1.607583e-04  5.082951e-10  6.777269e-10 -5.849685e-01 -1.792883e-08
   2.737746e-03  3.650328e-03  3.189132e-03]
 [-1.997171e-04 -1.075730e-08 -1.434306e-08 -1.792883e-08  2.412668e+00
  -9.645498e-05 -1.286066e-04 -1.607583e-04]
 [ 1.075730e-08  2.688693e-04  2.190197e-03  2.737746e-03 -9.645498e-05
  -5.849685e-01  4.066361e-10  5.082950e-10]
 [ 1.434306e-08  2.190197e-03  1.546484e-03  3.650328e-03 -1.286066e-04
   4.066361e-10 -5.849685e-01  6.777266e-10]
 [ 1.792883e-08  2.737746e-03  3.650328e-03  3.189132e-03 -1.607583e-04
   5.082950e-10  6.777266e-10 -5.849685e-01]]
[[ 2.412668e+00  9.645498e-05  1.286066e-04  1.607583e-04 -1.99

This algorithm is much more complicated to implement and it is doing something that should be avoided at all costs in Python: deeply nested for loops. While this algorithm has good scaling, we will soon observe a very large cost prefactor in its performance. We can compare the performance of these algorithms by setting up a sequence of Argon clusters of increasing size:

In [23]:
from timeit import default_timer as timer

def build_fcc_cluster(radius, lattice_constant):
    vec1 = np.array([lattice_constant, lattice_constant, 0.0])
    vec2 = np.array([lattice_constant, 0.0, lattice_constant])
    vec3 = np.array([0.0, lattice_constant, lattice_constant])
    max_index = int(radius/np.linalg.norm(vec1))
    atomic_coordinates = np.array([ i*vec1 + j*vec2 + k*vec3 for i in range(-max_index, max_index+1)
                                                             for j in range(-max_index, max_index+1)
                                                             for k in range(-max_index, max_index+1)
                                                             if np.linalg.norm(i*vec1 + j*vec2 + k*vec3) <= radius ])
    return atomic_coordinates

for radius in np.arange(10.0,40.0,10.0):
    coord = build_fcc_cluster(radius, 9.9)
    
    time1 = timer()
    density_matrix = calculate_atomic_density_matrix(coord)
    hamiltonian_matrix = calculate_hamiltonian_matrix(coord, model_parameters)
    interaction_matrix = calculate_interaction_matrix(coord, model_parameters)
    chi_tensor = calculate_chi_tensor(coord, model_parameters)
    time2 = timer()
    print(len(coord))
    print('base matrix construction time =',time2-time1)
    fock_matrix = calculate_fock_matrix(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)
    time3 = timer()
    print('slow Fock construction time =',time3-time2)
    fock_matrix = calculate_fock_matrix_fast(hamiltonian_matrix, interaction_matrix, density_matrix, model_parameters)
    time4 = timer()
    print('fast Fock construction time =',time4-time3)

1
base matrix construction time = 0.00022841200000001116
slow Fock construction time = 0.0008647119999998232
fast Fock construction time = 0.013775430000000144
19
base matrix construction time = 0.17833182700000005
slow Fock construction time = 0.01604422900000002
fast Fock construction time = 4.620355772
55
base matrix construction time = 1.3799168360000005
slow Fock construction time = 0.42496538400000006
fast Fock construction time = 36.751312869


We could improve the implementation of Fock matrix construction in Python by organizing much of the work as dense matrix-matrix multiplications on small blocks of the matrix, which involves more sophisticated bookkeeping. Another way to improve its implementation is simply to port it as-is to C or C++, which have very low performance overheads in nested for loops.

## (Extra Credit) 6. Semiempirical model parameterization

For a semiempirical model to be useful, it must be parameterized to fit a set of reference data, usually from experiments or high-accuracy quantum chemistry simulation data. The parameters in this project have already been fit to data, which was carried out using the fitting procedure implemented in this section. Data fitting is an optimization problem, and we can make use of pre-existing Python optimization tools by encapsulating the fitting process into a function that inputs a list of parameters and outputs a fitting error that we want to minimize. We begin with our reference data, which is the set of occupied orbital energies, Hartree-Fock binding energies, and MP2 binding energies from all-electron calculations of Argon dimers at various separation distances:

In [24]:
# distance, HF binding energy, MP2 binding energy, 6 occupied HF orbital energies
reference_dimer = [ [ 5.67, 0.0064861, -0.0037137, -0.620241, -0.595697, -0.595697, -0.584812, -0.584812, -0.561277 ],
                    [ 6.24, 0.0022351, -0.0020972, -0.608293, -0.593456, -0.593456, -0.587742, -0.587742, -0.573604 ],
                    [ 6.62, 0.0010879, -0.0014491, -0.603106, -0.592621, -0.592621, -0.588897, -0.588897, -0.578874 ],
                    [ 6.99, 0.0005255, -0.0010120, -0.599442, -0.592082, -0.592082, -0.589651, -0.589651, -0.582576 ],
                    [ 7.18, 0.0003642, -0.0008494, -0.598041, -0.591886, -0.591886, -0.589922, -0.589922, -0.583988 ],
                    [ 7.56, 0.0001737, -0.0006037, -0.595889, -0.591599, -0.591599, -0.590315, -0.590315, -0.586154 ],
                    [ 7.94, 0.0000818, -0.0004342, -0.594388, -0.591408, -0.591408, -0.590569, -0.590569, -0.587662 ],
                    [ 8.32, 0.0000377, -0.0003159, -0.593344, -0.591281, -0.591281, -0.590731, -0.590731, -0.588708 ],
                    [ 9.26, 0.0000047, -0.0001502, -0.591934, -0.591118, -0.591118, -0.590927, -0.590927, -0.590120 ],
                    [ 10.21, 0.000000, -0.0000757, -0.591378, -0.591059, -0.591059, -0.590993, -0.590993, -0.590675 ] ]

We then write a function that encapsulates all of the setup and solving from earlier in the notebook to generate model predictions of the reference data:

In [25]:
def calculate_reference_data(atomic_coordinates, model_parameters):
    '''Returns the occupied orbital, HF, & MP2 energies for the input list of atomic coordinates'''
    ndof = len(atomic_coordinates)*orbitals_per_atom

    energy_ion = calculate_energy_ion(atomic_coordinates)
    density_matrix = calculate_atomic_density_matrix(atomic_coordinates)

    hamiltonian_matrix = calculate_hamiltonian_matrix(atomic_coordinates, model_parameters)
    interaction_matrix = calculate_interaction_matrix(atomic_coordinates, model_parameters)
    chi_tensor = calculate_chi_tensor(atomic_coordinates, model_parameters)

    density_matrix, fock_matrix = scf_cycle(hamiltonian_matrix, interaction_matrix, density_matrix, chi_tensor)

    occupied_energy, virtual_energy, occupied_matrix, virtual_matrix = partition_orbitals(fock_matrix)
    energy_scf = calculate_energy_scf(hamiltonian_matrix, fock_matrix, density_matrix)
    energy_hf = energy_ion + energy_scf
    energy_mp2 = calculate_energy_mp2(fock_matrix, interaction_matrix, chi_tensor)

    return occupied_energy, energy_hf, energy_mp2

print(calculate_reference_data(np.array([[0.0,0.0,0.0]]),model_parameters))
print(calculate_reference_data(np.array([[0.0,0.0,0.0],[6.99,0.0,0.0]]),model_parameters))
print(calculate_reference_data(np.array([[0.0,0.0,0.0],[20.0,0.0,0.0]]),model_parameters))

(array([-0.585, -0.585, -0.585]), -8.955, -4.874785400924543e-06)
(array([-0.593432, -0.586461, -0.586461, -0.583458, -0.583458, -0.576486]), -17.90947438357523, -4.6898013306991235e-05)
(array([-0.585, -0.585, -0.585, -0.585, -0.585, -0.585]), -17.910000000000004, -9.817276154639703e-06)


We use this function inside yet another function that loops over the reference data, performs simulations on the reference geometries, and returns the root-mean-squared (RMS) deviation between the reference data and model predictions:

In [36]:
def model_error(reference_dimer, model_parameters):
    '''Returns the RMS error between the model & reference data for the input set of model parameters'''
    atom_occupied, atom_hf, atom_mp2 = calculate_reference_data(np.array([[0,0,0]]), model_parameters)
    rms_error = 0.0

    for data_list in reference_dimer:
        mol_coord = np.array([ [0,0,0], [data_list[0],0,0] ])
        mol_occupied, mol_hf, mol_mp2 = calculate_reference_data(mol_coord, model_parameters)
        mol_hf -= 2.0*atom_hf
        mol_mp2 -= 2.0*atom_mp2
        rms_error += (mol_hf - data_list[1])**2 + (mol_mp2 - data_list[2])**2
        rms_error += np.linalg.norm(mol_occupied - data_list[3:])**2

    print(np.sqrt(rms_error)/len(reference_dimer))
    return np.sqrt(rms_error)/len(reference_dimer)

print("rms error =", model_error(reference_dimer,model_parameters))

0.00028582914580843166
rms error = 0.00028582914580843166


Finally, we can use the SciPy optimizer to calculate the optimal model parameters:

In [32]:
import scipy.optimize

def objective_wrapper(param_list, reference_dimer, model_parameters, lock):
    '''Wraps model_error so that the model parameters are in a list'''
    new_parameters = {}
    i = 0
    for key in sorted(model_parameters.keys()):
        if lock != None and key in lock:
            new_parameters.update( { key : model_parameters[key] } )
        else:
            new_parameters.update( { key : param_list[i] } )
            i += 1
    return model_error(reference_dimer, new_parameters)

def fit_model(reference_dimer, model_parameters, lock=None):
    '''Returns optimized model parameters that best fit the reference data'''

    param_list = []
    for key in sorted(model_parameters.keys()):
        if lock != None and key in lock:
            pass
        else:
            param_list.append( model_parameters[key] )
    min_opt = { 'maxiter' : 2000 }
    result = scipy.optimize.minimize(objective_wrapper, param_list,
                                     (reference_dimer, model_parameters, lock),
                                     method='Nelder-Mead', options=min_opt)

    opt_parameters = {}
    i = 0
    for key in sorted(model_parameters.keys()):
        if lock != None and key in lock:
            opt_parameters.update( { key : model_parameters[key] } )
        else:
            opt_parameters.update( { key : result.x[i] } )
            i += 1
    return opt_parameters

new_parameters = fit_model(reference_dimer, model_parameters)
print("new rms error =", model_error(reference_dimer,new_parameters))
print(new_parameters)

0.0031810109536056404
0.003181006826907545
0.06674049999123469
0.0031871857189222814
0.09611844578269767
0.0031796720485551427
0.0035447614540679135
0.0031965153975676295
0.0031894633545379745
0.0031817839373530442
0.003181196848256282
0.003181011970587203
0.0031801747455470902
0.10143655508747391
0.0467304162935148
0.08033440044232223
0.029973024144694032
0.04592084768402997
0.022759667980623767
0.0319861154519794
0.014485783447621903
0.026150461126202517
0.010535763001150056
0.01853603123686283
0.006236531936618454
0.015301171444796532
0.004086332392921168
0.01136000768442105
0.0018598210505329474
0.00957871513665991
0.000735817880985868
0.002200457257822368
0.0015309230953770984
0.0012981725352233908
0.0009353451205790609
0.0006490346454602417
0.0011124482655967048
0.0005083130675297373
0.0017358097084492735
0.0007787445513166637
0.0013161953089342138
0.002006027922034818
0.0028339071588879884
0.0038082157666320185
0.001497753026501493
0.002895937607587337
0.001468397344623405
0.002

0.00029058865888484365
0.0002906412708170767
0.00029065636830574547
0.00029058274879602204
0.0002905378985985182
0.00029057829920597667
0.0002905459309814226
0.0002905735446218815
0.0002905555603529938
0.0002905432973689984
0.0002905307879509843
0.00029056702065664245
0.0002906487295224726
0.0002905100198432991
0.00029054608553594154
0.0002905060109683481
0.0002905416895131505
0.0002906462231632565
0.00029056455249011815
0.0002905236414813369
0.0002904813961428077
0.0002905155804939454
0.00029051307923064066
0.00029051374983190225
0.0002905304414863116
0.0002905075057342242
0.00029054270639112666
0.0002905239040891964
0.0002905226108575829
0.0002905212772300489
0.0002905264290554844
0.00029050836998736006
0.00029055379155746056
0.0002904974583573303
0.0002905190728239394
0.00029051991975802136
0.00029050556001567233
0.00029053735554236085
0.0002904948107180086
0.00029050996570115
0.00029053800366115245
0.0002904933030090794
0.00029051392463215177
0.00029049287661871284
0.00029049980929

0.0002903150002099989
0.0002903155365780043
0.00029031653169980695
0.0002903147239394846
0.00029031302818302975
0.00029031624383605854
0.00029031349825421246
0.0002903114627739725
0.00029031533268351184
0.0002903129557370333
0.0002903107755661129
0.0002903107565548198
0.00029031215888157124
0.00029031024721114896
0.0002903114523251474
0.00029031307050845134
0.0002903088470508633
0.0002903085576984515
0.00029030850092840886
0.00029030954728659385
0.0002903093063560256
0.0002903053639668391
0.00029030261112243693
0.00029030751389497747
0.00029030695586979526
0.0002903044073423702
0.0002903086731069829
0.00029030176375086287
0.0002902986152895884
0.0002903039791946504
0.0002903019527904807
0.0002903067173636605
0.000290297968781687
0.00029029383442162225
0.00029029904241483216
0.00029029903312068745
0.00029029467069095896
0.00029029655105856347
0.0002902920561675968
0.0002902885112612576
0.00029029148102920576
0.0002902946615406783
0.0002902862429953689
0.0002902799044430868
0.00029028499

0.00029028496924405397
0.0002901607168268751
0.00029016022061247303
0.00029028467449215405
0.00029016064212994023
0.0002901604755733558
0.00029028482946843234
0.00029016055168499985
0.0002901600513710036
0.000290160742204687
0.00029016042083755835
0.0002901600703159559
0.0002902845810352224
0.00029016046447867294
0.00029016035162618734
0.00029015976468906577
0.00029028405516014325
0.0002901597710773733
0.0002901602958347676
0.00029016001399330865
0.0002901600166835639
0.0002901595334511949
0.00029028379909015463
0.0002901596545812716
0.0002902846284022384
0.00029016005615245847
0.0002901599247425295
0.000290159865081624
0.00029015948279942415
0.00029015924726118387
0.0002901595232012365
0.00029015931875139947
0.0002901595163799739
0.0002901591829000838
0.0002901588796299691
0.0002902841701653022
0.0002901596681605223
0.00029015908198575647
0.00029015930438399014
0.0002901592627818026
0.0002901589962980744
0.0002901588393689811
0.0002901585936970375
0.00029015867008309957
0.000290158780

0.0002900568195099428
0.00029005125377454396
0.00029004895066095496
0.00029004699653220145
0.0002900435346541395
0.00029003588802058595
0.00029004034493973913
0.0002900528324946499
0.0002900435697590411
0.0002900410596742785
0.0002900385342077545
0.000290042192047826
0.00029003347375915987
0.0002900299968716704
0.0002900289665499525
0.0002900227889616188
0.0002900329448357282
0.00029003228978481373
0.00029002719273397406
0.00029002282048023944
0.0002900206850846508
0.00029001980422512836
0.00029001210848827704
0.00029000013567491423
0.00029002141667518785
0.00029001108570466115
0.00029001479549911906
0.00029000115414155083
0.0002900047569821044
0.0002900147351252589
0.00029000140051254795
0.0002900050755295808
0.0002899928766541276
0.0002899876869972758
0.0002900033350836719
0.00028999891329415967
0.00028998590119832076
0.0002899828696105324
0.0002899816026770697
0.00028998080773918696
0.00028997212142007006
0.0002899606015795183
0.0002899690740222105
0.00028996713542005497
0.000289973

0.00028627736346770703
0.0002862364112464356
0.0002863100963446191
0.00028624493966801935
0.0002862029340492848
0.00028619254803785013
0.000286284504783821
0.0002862376009270366
0.000286238859306291
0.0002862547597130593
0.00028623632842730113
0.00028631962640256897
0.0002862226289210289
0.0002862517946718158
0.0002861943416117889
0.000286209133155656
0.00028622891838112795
0.00028624906987776794
0.00028621796367936847
0.00028619725433024553
0.0002862167122380887
0.0002862098736510244
0.00028619703056294776
0.00028617743844120563
0.000286184400871174
0.0002862187624733123
0.0002861982403955567
0.00028617090964142813
0.0002861762738126036
0.00028617853331498234
0.0002862443972215226
0.00028618807869356397
0.0002861760582472708
0.00028619562512856094
0.00028617479057984973
0.0002861899078553361
0.00028616995162566533
0.00028619407215898726
0.0002861512531339647
0.00028614478902474203
0.00028615936640185815
0.0002861878512121886
0.00028615070659310026
0.0002861635315374625
0.0002861665280

0.0002858292443537739
0.0002858292907148843
0.0002858292334115994
0.00028585368581000956
0.0002858292404304869
0.0002858292446992848
0.00028582921222355864
0.0002858292461481676
0.0002858292238477285
0.00028582924181696004
0.00028582919645431113
0.0002858292357924628
0.00028582922125708354
0.00028585368153872154
0.00028582922319113065
0.00028582921939543674
0.00028582919294025685
0.00028585368421841665
0.0002858292170420448
0.00028582921456969483
0.00028585371363098187
0.0002858292153323333
0.0002858291962079096
0.00028582919161356
0.00028582917348784993
0.0002858536741503784
0.00028582921027886304
0.0002858292078355044
0.00028582917775951254
0.00028582917110763585
0.00028585370979608473
0.0002858291762679088
0.0002858536735230217
0.0002858291941174126
0.0002858536963546986
0.0002858291908915058
0.0002858291928498842
0.00028582918725982156
0.00028585370119475496
0.0002858291853755432
0.00028585367178927447
0.0002858291853394633
0.0002858291759018503
0.00028582920644313465
0.00028582917

Is this a good semiempirical model? Not really. It was designed to be simple in form and capture the correct qualitative behavior, but simplicity and accuracy are usually conflicting design principles. The model is not flexible enough to fit the data to very high accuracy, but its parameters also cannot be uniquely specified by the reference data because we aren't probing the system enough. We can identify this problem by examining the eigenvalues of the Hessian matrix for the objective function that we are minimizing to fit the model:

In [28]:
def model_hessian(reference_dimer, model_parameters, lock=None):
    param_list = []
    for key in sorted(model_parameters.keys()):
        if lock != None and key in lock:
            pass
        else:
            param_list.append( model_parameters[key] )
    num_param = len(param_list)
    hessian_matrix = np.zeros((num_param,num_param))
    dx = 0.01
    for i in range(num_param):
        for j in range(num_param):
            param_list[i] += dx
            param_list[j] += dx
            hes_pp = objective_wrapper(param_list, reference_dimer, model_parameters, lock)
            param_list[i] -= dx
            param_list[j] -= dx

            param_list[i] -= dx
            param_list[j] -= dx
            hes_mm = objective_wrapper(param_list, reference_dimer, model_parameters, lock)
            param_list[i] += dx
            param_list[j] += dx

            param_list[i] += dx
            param_list[j] -= dx
            hes_pm = objective_wrapper(param_list, reference_dimer, model_parameters, lock)
            param_list[i] -= dx
            param_list[j] += dx

            param_list[i] -= dx
            param_list[j] += dx
            hes_mp = objective_wrapper(param_list, reference_dimer, model_parameters, lock)
            param_list[i] += dx
            param_list[j] -= dx

            hessian_matrix[i,j] = (hes_pp - hes_mp - hes_pm + hes_mm)/(4*dx**2)
    return hessian_matrix

hessian_matrix = model_hessian(reference_dimer, new_parameters)
vals, vecs = np.linalg.eigh(hessian_matrix)
print('model Hessian eigenvalues:\n',vals)

0.0002921413638472155
0.0002932617411918774
0.00029033258598341454
0.00029033258598341454
0.03873073051626316
0.03873035204503046
0.038731144792176625
0.038731536072466495
0.00029109847696733704
0.0002916469325276741
0.00029012353941202626
0.0002909319723730905
0.007751120512122157
0.007750762265696301
0.007751527267035059
0.007751943570279285
0.00029042807153876484
0.0002912141443198207
0.00029061590455915754
0.0002912318572712527
0.0003063587801876048
0.0003067454952938116
0.00030537755080597257
0.0003063652965008084
0.0002924051824280439
0.0002929958378988528
0.00029232154139998925
0.0002931763106913508
0.0007303808931206701
0.0007311673964413819
0.0007332362719101992
0.0007330060525490378
0.000993286849863391
0.000993520618880767
0.0009933134017845618
0.0009934904566350007
0.0002904356209208753
0.00029156789073021843
0.0002908766530098354
0.0002915397863433621
0.00029052651538152716
0.000291216563429369
0.0002905143388544208
0.00029122737777256125
0.0003261924332598089
0.0003235169

0.0007446350981027251
0.0007467939692758459
0.0009932868498633719
0.0009935206188808302
0.0009934904566349949
0.0009933134017845607
0.038742711832351874
0.03874245322014421
0.03874245249345667
0.038742712563099566
0.0009932608607338543
0.0009932757970620098
0.0009932469643310826
0.0009932888730591418
0.007809428450940398
0.00780917264158804
0.007809169008464503
0.007809432048211083
0.0009932206002734889
0.000993270409302213
0.0009932421259315136
0.0009932491651998723
0.0009911710609581236
0.0009612693972845778
0.0010039105519469263
0.0010343222269440658
0.0009937885924093076
0.000993770050690185
0.0009937400094887463
0.0009938153368558738
0.0011988768943646457
0.0011991878054904933
0.0011991643028758697
0.0011989004838820045
0.0008053623248675401
0.0019218016452636614
0.0002903325859834549
0.0002903325859834549
0.0009932694841786503
0.0009933473241659043
0.0009933316029441513
0.0009933101599391213
0.0009932326754463745
0.000993257745725115
0.0009932293061191678
0.0009932610849052925
0.

Small eigenvalues correspond to directions in parameter space where we can change parameters without changing how well the model fits the data. This problem is not specific to our simple QM project - semiempirical model building has persistent problems with model indeterminacy as they add more parameters and details to improve accuracy. Because these models have historically be used to reproduce a limited set of observables (e.g. heat of formation), they've been limited in what reference data they can use for model fitting.

We can use this same model and fitting process for other noble gases such as Neon by applying it to a different set of reference data:

In [38]:
neon_ref_data = [
[ 5.29301 , 0.000356767 , -0.000275745 , -0.857733 , -0.851311 , -0.851311 , -0.849117 , -0.849117 , -0.843045 ],
[ 5.48204 , 0.000222991 , -0.000229153 , -0.856225 , -0.851119 , -0.851119 , -0.849443 , -0.849443 , -0.844581 ],
[ 5.67108 , 0.000138635 , -0.000190295 , -0.855025 , -0.850968 , -0.850968 , -0.849687 , -0.849687 , -0.845800 ],
[ 5.86011 , 8.54899e-05 , -0.000157984 , -0.854072 , -0.850849 , -0.850849 , -0.849869 , -0.849869 , -0.846765 ],
[ 6.04915 , 5.21333e-05 , -0.000131269 , -0.853315 , -0.850757 , -0.850757 , -0.850006 , -0.850006 , -0.847529 ],
[ 6.23819 , 3.13588e-05 , -0.000109318 , -0.852714 , -0.850684 , -0.850684 , -0.850108 , -0.850108 , -0.848135 ],
[ 6.42722 , 1.85686e-05 , -9.13551e-05 , -0.852237 , -0.850626 , -0.850626 , -0.850185 , -0.850185 , -0.848614 ],
[ 6.61626 , 1.07945e-05 , -7.66555e-05 , -0.851859 , -0.850582 , -0.850582 , -0.850244 , -0.850244 , -0.848993 ],
[ 6.80529 , 6.11193e-06 , -6.45702e-05 , -0.851559 , -0.850547 , -0.850547 , -0.850287 , -0.850287 , -0.849293 ],
[ 6.99433 , 3.28677e-06 , -5.45556e-05 , -0.851322 , -0.850520 , -0.850520 , -0.850321 , -0.850321 , -0.849531 ],
[ 7.18336 , 1.55256e-06 , -4.61875e-05 , -0.851134 , -0.850499 , -0.850499 , -0.850346 , -0.850346 , -0.849719 ],
[ 7.3724 , 4.57149e-07 , -3.91539e-05 , -0.850986 , -0.850483 , -0.850483 , -0.850366 , -0.850366 , -0.849868 ],
[ 7.56144 , -2.50044e-07 , -3.3231e-05 , -0.850869 , -0.850471 , -0.850471 , -0.850380 , -0.850380 , -0.849986 ],
[ 7.75047 , -7.01547e-07 , -2.82527e-05 , -0.850776 , -0.850461 , -0.850461 , -0.850392 , -0.850392 , -0.850079 ],
[ 7.93951 , -9.693e-07 , -2.40853e-05 , -0.850703 , -0.850454 , -0.850454 , -0.850400 , -0.850400 , -0.850153 ],
[ 8.12854 , -1.09914e-06 , -2.06105e-05 , -0.850645 , -0.850448 , -0.850448 , -0.850407 , -0.850407 , -0.850211 ],
[ 8.31758 , -1.12749e-06 , -1.7719e-05 , -0.850600 , -0.850444 , -0.850444 , -0.850412 , -0.850412 , -0.850257 ],
[ 8.50662 , -1.08659e-06 , -1.53105e-05 , -0.850564 , -0.850440 , -0.850440 , -0.850416 , -0.850416 , -0.850294 ],
[ 8.69565 , -1.00439e-06 , -1.32963e-05 , -0.850536 , -0.850438 , -0.850438 , -0.850419 , -0.850419 , -0.850322 ],
[ 8.88469 , -9.0295e-07 , -1.16011e-05 , -0.850513 , -0.850436 , -0.850436 , -0.850421 , -0.850421 , -0.850345 ],
[ 9.07372 , -7.97673e-07 , -1.01637e-05 , -0.850495 , -0.850434 , -0.850434 , -0.850423 , -0.850423 , -0.850363 ],
[ 9.26276 , -6.977e-07 , -8.93627e-06 , -0.850482 , -0.850433 , -0.850433 , -0.850425 , -0.850425 , -0.850377 ],
[ 9.4518 , -6.07201e-07 , -7.88179e-06 , -0.850470 , -0.850432 , -0.850432 , -0.850426 , -0.850426 , -0.850389 ],
[ 9.64083 , -5.27071e-07 , -6.97177e-06 , -0.850462 , -0.850432 , -0.850432 , -0.850427 , -0.850427 , -0.850398 ],
[ 9.82987 , -4.56455e-07 , -6.18389e-06 , -0.850455 , -0.850431 , -0.850431 , -0.850428 , -0.850428 , -0.850405 ],
[ 10.0189 , -3.93899e-07 , -5.50014e-06 , -0.850450 , -0.850431 , -0.850431 , -0.850428 , -0.850428 , -0.850410 ],
[ 10.2079 , -3.38041e-07 , -4.90551e-06 , -0.850445 , -0.850431 , -0.850431 , -0.850429 , -0.850429 , -0.850415 ],
[ 10.397 , -2.87888e-07 , -4.38722e-06 , -0.850442 , -0.850431 , -0.850431 , -0.850429 , -0.850429 , -0.850418 ],
[ 10.586 , -2.42876e-07 , -3.93422e-06 , -0.850439 , -0.850431 , -0.850431 , -0.850429 , -0.850429 , -0.850421 ],
[ 10.775 , -2.02724e-07 , -3.53699e-06 , -0.850437 , -0.850430 , -0.850430 , -0.850429 , -0.850429 , -0.850423 ]
]

neon_param = fit_model(neon_ref_data, model_parameters)
print("neon rms error =", model_error(neon_ref_data,neon_param))
print(neon_param)

#completed output saved for posterity:
neon_param = {'coulomb_p': -0.010255409806855187,
              'coulomb_s': 0.4536486561938202,
              'dipole': 1.6692376991516769,
              'energy_p': -3.1186533988406335,
              'energy_s': 11.334912902362603,
              'r_hop': 2.739689713337267,
              'r_pseudo': 1.1800779720963734,
              't_pp1': -0.029546671673199854,
              't_pp2': -0.0041958662271044875,
              't_sp': 0.000450562836426027,
              't_ss': 0.0289251941290921,
              'v_pseudo': -0.015945813280635074}

0.11610651757707637
0.11610652027934001
0.156384549956229
0.11610313423460357
0.06264065153161538
0.11610749835175994
0.11613170575490957
0.11612109915431615
0.11611106139349571
0.1161067991141531
0.11610640640172393
0.11610651647952806
0.11610743817931761
0.0669412900378389
0.09897816454862517
0.09613302112969889
0.09280770748094927
0.08892876312233945
0.08439957288695157
0.07911634560391152
0.07295286682439096
0.06576256987544467
0.05737537337937533
0.028051800602614158
0.04270412942221838
0.030493989360646703
0.03622753153868676
0.02906766704630541


KeyboardInterrupt: 